In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.optimize import fsolve, brentq, minimize_scalar
from scipy.interpolate import interp1d
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Markdown, Image

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6), 'figure.dpi': 130,
                     'axes.titlesize': 'x-large', 'axes.labelsize': 'large',
                     'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for advanced Overlapping Generations (OLG) model analysis.")

# Part 4: Macroeconomic Models
## Chapter 4.04: Overlapping Generations (OLG) Models: Life Cycles, Debt, and Social Security

### Table of Contents
1.  [The Diamond (1965) Two-Period OLG Model](#1.-The-Diamond-(1965)-Two-Period-OLG-Model)
    *   [1.1 The Household's Problem and the Law of Motion](#1.1-The-Household's-Problem-and-the-Law-of-Motion)
    *   [1.2 Dynamic Inefficiency](#1.2-Dynamic-Inefficiency)
2.  [Government Debt and Ricardian Equivalence](#2.-Government-Debt-and-Ricardian-Equivalence)
    *   [2.1 Failure of Ricardian Equivalence in the OLG Model](#2.1-Failure-of-Ricardian-Equivalence-in-the-OLG-Model)
    *   [2.2 Restoring Equivalence with Altruistic Bequests](#2.2-Restoring-Equivalence-with-Altruistic-Bequests)
3.  [A Multi-Period Life-Cycle Model](#3.-A-Multi-Period-Life-Cycle-Model)
    *   [3.1 The Model Environment](#3.1-The-Model-Environment)
    *   [3.2 Solving via Backward Induction](#3.2-Solving-via-Backward-Induction)
4.  [The Auerbach-Kotlikoff Model for Policy Analysis](#4.-The-Auerbach-Kotlikoff-Model-for-Policy-Analysis)
5.  [Chapter Summary](#5.-Chapter-Summary)
6.  [Exercises](#6.-Exercises)

### Introduction: Beyond the Infinitely-Lived Agent

The macroeconomic models studied thus far (RCK, RBC) rely on the powerful but abstract simplification of a single, infinitely-lived representative household. This abstraction omits a fundamental feature of economic life: the **life cycle**. Individuals are born, work, save for retirement, and eventually die. The economy at any point in time is a heterogeneous mix of overlapping generations, each at a different stage of life. The interactions between these generations—the young saving while the old dissave—are crucial for understanding a host of major economic issues.

**Overlapping Generations (OLG) models**, pioneered by Maurice Allais (1947), Paul Samuelson (1958), and Peter Diamond (1965), provide the canonical framework for studying these life-cycle dynamics. By replacing the infinite horizon of the representative agent with a finite, more realistic lifetime, these models build a bridge between microeconomic life-cycle theory and aggregate macroeconomic outcomes. This seemingly simple change has profound implications:

1.  **Failure of Ricardian Equivalence:** In OLG models, government debt is not neutral because its burden can be shifted to future, unborn generations.
2.  **Possibility of Dynamic Inefficiency:** An OLG economy can over-accumulate capital, leading to a situation where a policy intervention like Social Security can create a Pareto improvement.

This notebook develops the OLG model from its foundations to several advanced extensions, showing its power and flexibility.

### 1. The Diamond (1965) Two-Period OLG Model

#### 1.1 The Household's Problem and the Law of Motion
The economy is populated by a series of overlapping generations who live for two periods (young and old). The young supply labor, consume, and save. The old do not work and consume their savings. The household's problem is to choose consumption in both periods ($c_1, c_2$) to maximize lifetime utility $U = u(c_1) + \beta u(c_2)$ subject to their budget constraints.

This leads to a saving function $s_t = s(w_t, r_{t+1})$. The aggregate capital stock in $t+1$ is the total saving of the young in period $t$: $K_{t+1} = N_t s_t$. In per-worker terms, this gives the law of motion for capital per worker: $k_{t+1} = s(w_t, r_{t+1}) / (1+n)$. With competitive markets, factor prices equal marginal products, so $w_t = w(k_t)$ and $r_{t+1} = r(k_{t+1})$. This yields an implicit difference equation for capital:
$$ k_{t+1}(1+n) = s(w(k_t), r(k_{t+1})) $$

In [ ]:
sec("Solving the General OLG Model")

class GeneralOLGModel:
    """
    Implements the two-period Diamond (1965) OLG model.
    Parameters:
      - A: Total factor productivity
      - alpha: Capital share
      - beta: Subjective discount factor
      - n: Population growth rate
      - sigma: Coefficient of relative risk aversion
    """
    def __init__(self, A=10.0, alpha=0.33, beta=0.96, n=0.01, sigma=2.0):
        self.A, self.alpha, self.beta, self.n, self.sigma = A, alpha, beta, n, sigma
        self.prod_f = lambda k: self.A * k**self.alpha
        self.wage_f = lambda k: (1 - self.alpha) * self.prod_f(k)
        self.interest_f = lambda k: self.alpha * self.A * k**(self.alpha - 1)
        self.k_star = self._solve_steady_state()

    def _law_of_motion_implicit(self, k_next, k_t):
        w_t = self.wage_f(k_t); r_next = self.interest_f(k_next)
        saving_lhs = k_next * (1 + self.n)
        saving_rhs = w_t / (1 + (self.beta * (1 + r_next))**(-1/self.sigma))
        return saving_lhs - saving_rhs

    def _solve_steady_state(self): return brentq(lambda k: self._law_of_motion_implicit(k, k), 0.1, 50)
    def solve_for_k_next(self, k_t): return brentq(lambda k_next: self._law_of_motion_implicit(k_next, k_t), 0.1, 50)

olg_gen = GeneralOLGModel(sigma=2.0)
k_grid = np.linspace(0.5, olg_gen.k_star * 1.5, 100)
k_next_path = [olg_gen.solve_for_k_next(k) for k in k_grid]

fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(k_grid, k_next_path, lw=2.5, label='$k_{t+1} = g(k_t)$')
ax.plot(k_grid, k_grid, 'k--', lw=1.5, label='$k_{t+1} = k_t$')
ax.plot(olg_gen.k_star, olg_gen.k_star, 'ro', ms=12, label=f'Steady State $k^*$')
ax.set_title('Dynamics in the General OLG Model', fontsize=16)
ax.set_xlabel('$k_t$'); ax.set_ylabel('$k_{t+1}$'); ax.legend()
plt.show()

#### 1.2 Dynamic Inefficiency
A striking and counter-intuitive result from OLG models is that the decentralized market equilibrium can be **dynamically inefficient**. This is a situation where the economy has accumulated *too much* capital, saving so much that it has pushed the marginal product of capital (the interest rate) below the economy's growth rate ($r^* < n$). The economy is investing in capital that has a lower rate of return than the natural growth of the system itself.

In such a scenario, it is possible to make everyone better off—a Pareto improvement. The intuition is that a policy which forces the current young generation to save less (e.g., a mandatory social security tax that funds a transfer to the current old) will reduce the capital stock. This moves the economy from the inefficiently high $k^*$ back towards the **Golden Rule** level of capital ($k_{gold}$) which maximizes steady-state consumption. The initial old generation is better off (they receive a transfer), and all future generations are better off because they are born into an economy with a more efficient capital stock that can deliver higher consumption.

If an economy is dynamically inefficient, a **Pay-As-You-Go (PAYG) Social Security system** can create a Pareto improvement. By taxing the young and transferring the proceeds to the old, the government reduces private saving. This reduces the steady-state capital stock from the inefficient $k^*$ towards the efficient $k_{gold}$, which can raise the consumption of *all* generations.

If an economy is dynamically inefficient, a **Pay-As-You-Go (PAYG) Social Security system** can create a Pareto improvement. By taxing the young and transferring the proceeds to the old, the government reduces private saving. This reduces the steady-state capital stock from the inefficient $k^*$ towards the efficient $k_{gold}$, which can raise the consumption of *all* generations.


### 2. Government Debt and Ricardian Equivalence

#### 2.1 Failure of Ricardian Equivalence in the OLG Model
The **Ricardian Equivalence proposition**, named after David Ricardo and famously revived by Robert Barro, states that a government's choice between financing spending with debt or with taxes has no effect on consumption or national saving. The logic is that rational, forward-looking agents understand that government borrowing today is simply a promise of higher taxes in the future. To prepare for that future tax liability, they will save the entirety of any debt-financed tax cut. The timing of taxes doesn't matter, only their present value.

This proposition **fails** in the standard OLG model. The key reason is the finite lifetime of the agents. If the government issues debt to give a tax cut to the current young generation, the debt will be repaid by taxes levied on generations that are not yet born. The current generation receives a windfall that they do not have to pay for. Since they are disconnected from the future taxpayers, they treat the tax cut as an increase in their lifetime wealth and increase their consumption. This raises aggregate demand and lowers national saving, meaning the policy is not neutral.

#### 2.2 Restoring Equivalence with Altruistic Bequests
**Barro (1974)** showed that Ricardian Equivalence could be restored if generations were linked by **operative altruistic bequests**. If parents care about their children's utility and are already planning to leave them a bequest, they will respond to a debt-financed tax cut by simply increasing their bequest to their children by the full amount of the future tax liability. The tax cut is fully saved, national saving is unchanged, and the policy is neutral. The family dynasty acts as a single, infinitely-lived agent.

### 3. A Multi-Period Life-Cycle Model
The two-period model is a powerful theoretical tool, but for quantitative work, we need a more realistic **multi-period life-cycle model**. We can model households that live for $J$ periods, with a realistic hump-shaped earnings profile over their working years and a retirement phase.

In [ ]:
sec("Solving a Multi-Period Life-Cycle Model")
# --- Model Parameters ---
# J: Total lifetime periods (e.g., 80 years)
# J_R: Retirement age (e.g., 65)
# BETA_LC: Discount factor
# SIGMA_LC: Coefficient of relative risk aversion
# R_LC: Risk-free interest rate
# A_GRID_LC: Grid of possible asset holdings
J = 80
J_R = 65
BETA_LC = 0.96
SIGMA_LC = 2.0
R_LC = 0.04
A_GRID_LC = np.linspace(0, 500, 100)

def u_lc(c): 
    # Add a small epsilon to avoid division by zero
    return (np.maximum(c, 1e-9)**(1 - SIGMA_LC) - 1) / (1 - SIGMA_LC)

# Realistic hump-shaped earnings profile
def earnings_profile(age):
    if age < J_R: return 2 * (age - 18) - 0.02 * (age - 18)**2 + 20
    else: return 0 # No income in retirement
Y_PROFILE = np.array([earnings_profile(j) for j in range(J)])

# --- Solve via Backward Induction ---
V = np.zeros((J, len(A_GRID_LC)))
C_policy = np.zeros_like(V)

# Last period: consume all assets
C_policy[J-1, :] = A_GRID_LC * (1+R_LC)
V[J-1, :] = u_lc(C_policy[J-1, :])

for j in range(J - 2, -1, -1):
    V_next_interp = interp1d(A_GRID_LC, V[j+1, :], bounds_error=False, fill_value="extrapolate")
    for i, a in enumerate(A_GRID_LC):
        budget = a * (1+R_LC) + Y_PROFILE[j]
        if budget <= 1e-6:
            # If budget is effectively zero, consume zero and leave zero assets.
            C_policy[j, i] = budget
            V[j, i] = u_lc(1e-9) + BETA_LC * V_next_interp(0.0)
        else:
            def objective(c):
                a_next = budget - c
                return -(u_lc(c) + BETA_LC * V_next_interp(a_next))
            res = minimize_scalar(objective, bounds=(1e-6, budget), method='bounded')
            V[j, i] = -res.fun
            C_policy[j, i] = res.x

note("Solved the multi-period life-cycle model.")

fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(A_GRID_LC, np.arange(J))
ax.plot_surface(X, Y, C_policy, cmap='viridis')
ax.set_title('Life-Cycle Consumption Policy c(age, assets)')
ax.set_xlabel('Assets'); ax.set_ylabel('Age'); ax.set_zlabel('Consumption')
ax.view_init(elev=30, azim=-135)
plt.show()

### 4. The Auerbach-Kotlikoff Model for Policy Analysis
The **Auerbach-Kotlikoff (AK) model** is the canonical large-scale computational OLG model used for quantitative policy analysis. It extends the life-cycle model to a full general equilibrium setting, where wages and interest rates are determined endogenously. The model typically features:
- 55 overlapping generations (from age 21 to 75).
- A realistic life-cycle earnings profile.
- A detailed government sector with various distortionary taxes (on labor, capital, consumption) and transfer systems (like Social Security).

Solving an AK model is computationally intensive. It requires finding the equilibrium path of prices that is consistent with the optimal life-cycle plans of all 55 generations. The standard solution method is an iterative algorithm (using Gauss-Seidel or similar methods) to find the sequence of factor prices that clears the capital and labor markets in every period along the transition path after a policy change.

Despite the complexity, these models are the primary tool for analyzing the long-run and transitional effects of major fiscal reforms, such as switching from an income tax to a consumption tax, or reforming the Social Security system.

### 5. Chapter Summary
- **Finite Lives Matter:** OLG models, by incorporating finite lifetimes, generate results that are fundamentally different from infinite-horizon representative-agent models.
- **Ricardian Equivalence Fails:** Government debt is not neutral in OLG models, as its burden can be shifted to future generations. This link can be restored through **altruistic bequests**.
- **Dynamic Inefficiency:** OLG models can feature over-accumulation of capital ($r < n$), a situation where a PAYG Social Security system can be Pareto-improving.
- **Life-Cycle Models:** Multi-period OLG models with realistic earnings profiles are the workhorse for studying savings, retirement, and the effects of fiscal policy.
- **Auerbach-Kotlikoff Models:** These are large-scale computational OLG models used for detailed quantitative analysis of major policy reforms.

### 6. Exercises

1.  **Dynamic Inefficiency Condition:** For the general OLG model with CRRA utility, derive the condition on the parameters ($\\alpha, \beta, n, \sigma$) that determines whether the economy is dynamically efficient or inefficient. For the baseline parameters, is the model dynamically efficient?

2.  **Bequest Motive:** In the Barro-Ricardo model, under what conditions might the bequest motive be "inoperative"? What would this imply for the validity of Ricardian Equivalence?

3.  **Simulating a Life Path:** Using the solved policy functions from the multi-period life-cycle model, simulate and plot the path of consumption, assets, and income for a single household over their entire lifetime.

4.  **Labor Supply in the Life-Cycle:** Extend the multi-period life-cycle model to include an endogenous labor supply choice. How would you expect the optimal path of hours worked to look over the life cycle? Why?

5.  **Policy Analysis with OLG:** A government proposes to increase retirement benefits in its PAYG Social Security system, financing it with a higher payroll tax on the young. Using the logic of the OLG model, trace out the likely general equilibrium effects on the steady-state capital stock, wages, and interest rates.